In [ ]:
from urllib.parse import urlparse, unquote
from pathlib import Path
import io

import requests
import numpy as np
import pandas as pd
import geopandas

# Tools

In [ ]:
def download_file(url, dir=None, fname=None, overwrite=False, verbose=True):
    """Download file from given `url` and put it into `dir`.
    Current working directory is used as default. Missing directories are created.
    File name from `url` is used as default.
    Return absolute pathlib.Path of the downloaded file."""
    
    if dir is None:
        dir = '.'
    dpath = Path(dir).resolve()
    dpath.mkdir(parents=True, exist_ok=True)

    if fname is None:
        fname = Path(urlparse(url).path).name
    fpath = dpath / fname
    
    if not overwrite and fpath.exists():
        print(f'File {fname} already exists.')
        return fpath

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(fpath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=2**20):
                f.write(chunk)
    
    if verbose: print(f'Download complete: {fname}.')
    return fpath

# Geography

[geocodes](https://www.census.gov/geographies/reference-files/2019/demo/popest/2019-fips.html)

In [ ]:
f = download_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip', 'data')
df = geopandas.read_file(f)
df = df.rename(columns={'STATEFP': 'st', 'NAME': 'name'})
df = df[['st', 'name', 'geometry']]
df['cty'] = '000'
st = df

f = download_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_20m.zip', 'data')
df = geopandas.read_file(f)
df = df.rename(columns={'STATEFP': 'st', 'COUNTYFP': 'cty', 'NAME': 'name_cty'})
df = df[['st', 'cty', 'name_cty', 'geometry']]
df = df.merge(st[['st', 'name']], 'left')
df['name'] = df['name_cty'] + ' county, ' + df['name']
del df['name_cty']

df = pd.concat([df, st]).sort_values(['st', 'cty'], ignore_index=True)
df = df[['st', 'cty', 'name', 'geometry']]
geo = df

# Population

[home](https://www.census.gov/programs-surveys/popest/data/data-sets.html)
[2000-2010](https://www.census.gov/data/datasets/time-series/demo/popest/intercensal-2000-2010-counties.html)
[2010-2019](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html)

For years before 2000 I could not find data in convenient format.

[Character encoding](https://www.census.gov/programs-surveys/geography/technical-documentation/user-note/special-characters.html): newer files use "UTF-8", older use "ISO-8859-1".

Also check this: Population, Population Change, and Estimated Components of Population Change: April 1, 2010 to July 1, 2019 (CO-EST2019-alldata)

In [ ]:
f = download_file('https://www2.census.gov/programs-surveys/popest/tables/1990-2000/estimates-and-change-1990-2000/2000c8_00.txt', 'data')
with open(f, encoding='ISO-8859-1') as file:
    data = io.StringIO()
    in_table = False
    for line in file:
        if in_table:
            if line[0] == '1':
                data.write(line)
            else:
                break
        else:
            if line[0] == '1':
                in_table = True
                data.write(line)

data.seek(0)
df = pd.read_fwf(data, dtype='str', header=None, columns=cols)
# skip first row (US total), keep fips and popest cols
df = df.iloc[1:, 1:13]
df.columns = ['fips'] + [f'pop{y}' for y in range(2000, 1989, -1)]
df['fips'] = df['fips'].str.pad(5, 'right', '0')
df['st'] = df['fips'].str[:2]
df['cty'] = df['fips'].str[2:]
df = df.drop(columns=['pop2000', 'fips'])
df = pd.wide_to_long(df, 'pop', ['st', 'cty'], 'year')
df = df.reset_index()
df['pop'] = pd.to_numeric(df['pop'].str.replace(',', '', regex=False)).astype('int')

d1 = df

In [ ]:
f = download_file('https://www2.census.gov/programs-surveys/popest/datasets/2000-2010/intercensal/county/co-est00int-tot.csv', 'data')
cols = ['STATE', 'COUNTY'] + [f'POPESTIMATE{y}' for y in range(2000, 2010)]
df = pd.read_csv(f, encoding='ISO-8859-1', dtype='str', usecols=cols)
df = pd.wide_to_long(df, 'POPESTIMATE', ['STATE', 'COUNTY'], 'year')
df = df.reset_index().rename(columns={'STATE': 'st', 'COUNTY': 'cty', 'POPESTIMATE': 'pop'})
df['st'] = df['st'].str.pad(2, fillchar='0')
df['cty'] = df['cty'].str.pad(3, fillchar='0')
df['pop'] = df['pop'].astype('int')
d2 = df

In [ ]:
f = download_file('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv', 'data')
cols = ['STATE', 'COUNTY'] + [f'POPESTIMATE{y}' for y in range(2010, 2020)]
df = pd.read_csv(f, encoding='ISO-8859-1', dtype='str', usecols=cols)
df = pd.wide_to_long(df, 'POPESTIMATE', ['STATE', 'COUNTY'], 'year')
df = df.reset_index().rename(columns={'STATE': 'st', 'COUNTY': 'cty', 'POPESTIMATE': 'pop'})
df['pop'] = df['pop'].astype('int')
d3 = df

In [ ]:
df = pd.concat([d1, d2, d3], ignore_index=True)

d = df.query('cty == "000"').groupby('year')['pop'].sum()
d = d.to_frame('pop').reset_index()
d[['st', 'cty']] = ['00', '000']
df = pd.concat([df, d], ignore_index=True)

df = df.sort_values('year')
df['pop_'] = df.groupby(['st', 'cty'])['pop'].shift()
df['pop_gr'] = df.eval('(pop / pop_ - 1) * 100')
del df['pop_']

df = df.sort_values(['st', 'cty', 'year'])
df = df[['st', 'cty', 'year', 'pop', 'pop_gr']]
pop = df

Something strange happens in 2000.

In [ ]:
pop.query('st == "00" and cty == "000"').set_index('year')['pop'].plot()

In [ ]:
pop.query('st == "01" and cty == "000"').set_index('year')['pop'].plot()

# Employment

[datasets](https://www.census.gov/data/datasets/time-series/econ/bds/bds-datasets.html)

In [ ]:
# economy-wide
f = download_file('https://www2.census.gov/programs-surveys/bds/tables/time-series/bds2019.csv', 'data')
df = pd.read_csv(f, usecols=['year', 'emp', 'net_job_creation_rate'], dtype='str')
df[['st', 'cty']] = ['00', '000']
d1 = df

# by state
f = download_file('https://www2.census.gov/programs-surveys/bds/tables/time-series/bds2019_st.csv', 'data')
df = pd.read_csv(f, usecols=['year', 'st', 'emp', 'net_job_creation_rate'], dtype='str')
df['cty'] = '000'
d2 = df

# by county
f = download_file('https://www2.census.gov/programs-surveys/bds/tables/time-series/bds2019_cty.csv', 'data')
df = pd.read_csv(f, usecols=['year', 'st', 'cty', 'emp', 'net_job_creation_rate'], dtype='str')

df = pd.concat([d1, d2, df], ignore_index=True)
df = df.rename(columns={'net_job_creation_rate': 'emp_gr'})
df['year'] = df['year'].astype('int16')
df['emp'] = pd.to_numeric(df['emp'], 'coerce').astype('Int32')
df['emp_gr'] = pd.to_numeric(df['emp_gr'], 'coerce').astype('Float32')
df = df[['st', 'cty', 'year', 'emp', 'emp_gr']]
emp = df

## API

[BDS API](https://www.census.gov/data/developers/data-sets/business-dynamics.html)

In [ ]:
key = open('census_api_key.txt').read()
url = 'https://api.census.gov/data/timeseries/bds'
st = '55'
r = requests.get(f'{url}?get=NAME,ESTAB,EMP,YEAR&for=county:*&in=state:{st}&time=from+2015+to+2019&NAICS=00&key={key}')
d = r.json()

In [ ]:
df = pd.DataFrame(d[1:], columns=d[0])
df.query('county == "025"')

# pop vs emp

In [ ]:
df = pop.merge(emp, 'left')
data_yr = df

In [ ]:
def data_ag(y0, y1):
    d = data_yr.query('year == @y0 or year == @y1').set_index(['st', 'cty', 'year'])[['pop', 'emp']].unstack('year')
    d = d[(d.notna() & (d > 0)).all(1)]
    d = d.stack(0)
    d = np.power(d[y1] / d[y0], 1/(y1-y0+1)).unstack().add_suffix('_agr_abs')
    d = (d - 1) * 100
    d = d.reset_index()

    d1 = d.query('cty == "000"').rename(columns={'pop_agr_abs': 'ref_pop_agr', 'emp_agr_abs': 'ref_emp_agr'})
    d = d.merge(d1.drop(columns='cty'), 'left')
    d.loc[d['cty'] == '000', ['ref_pop_agr', 'ref_emp_agr']] = d.loc[d['st'] == '00', ['ref_pop_agr', 'ref_emp_agr']].values
    d['pop_agr_rel'] = d['pop_agr_abs'] - d['ref_pop_agr']
    d['emp_agr_rel'] = d['emp_agr_abs'] - d['ref_emp_agr']
    return d

In [ ]:
def color_from_gr(df, abs_rel):
    e = df['emp_agr_' + abs_rel]
    p = df['pop_agr_' + abs_rel]
    x = pd.Series(index=df.index, dtype='str')
    x[(p >= 0) & (e >= 0)] = 'red'
    x[(p >= 0) & (e <  0)] = 'green'
    x[(p <  0) & (e >= 0)] = 'orange'
    x[(p <  0) & (e <  0)] = 'blue'
    return x

In [ ]:
d = data_ag(2005, 2012).query('cty == "000"').copy()
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
d['abs_cat'] = color_from_gr(d, 'abs')
d['rel_cat'] = color_from_gr(d, 'rel')

ax = axes[0]
d.plot.scatter('pop_agr_abs', 'emp_agr_abs', ax=ax, c='abs_cat')
xlim = abs(max(ax.get_xlim(), key=abs))
ax.set_xlim(-xlim, xlim)
ylim = abs(max(ax.get_ylim(), key=abs))
ax.set_ylim(-ylim, ylim)
ax.axvline(0, ls='-')
ax.axhline(0, ls='-')
ax.axvline(d['ref_pop_agr'].iloc[0], ls=':')
ax.axhline(d['ref_emp_agr'].iloc[0], ls=':')

ax = axes[1]
d.plot.scatter('pop_agr_rel', 'emp_agr_rel', ax=ax, c='rel_cat')
xlim = abs(max(ax.get_xlim(), key=abs))
ax.set_xlim(-xlim, xlim)
ylim = abs(max(ax.get_ylim(), key=abs))
ax.set_ylim(-ylim, ylim)
ax.axvline(0, ls='-')
ax.axhline(0, ls='-')

# Dashboard

GeoData is a natural choise for area layer, but it does not support `style_callback` ([GH PR](https://github.com/jupyter-widgets/ipyleaflet/pull/786)). Using GeoJSON instead.

In [ ]:
import ipywidgets as widgets
import ipyleaflet as leaflet
import matplotlib.pyplot as plt
import json

In [ ]:
def data_map(st, y0, y1, abs_rel):
    if st == '00':
        df = geo.query('cty == "000"')
    else:
        df = geo.query('st == @st')

    df = df.merge(data_ag(y0, y1))
    df['color'] = color_from_gr(df, abs_rel)
    df = df[['st', 'cty', 'name', 'geometry', 'color']]
    return df

def click_geo(**kw):
    p = kw['properties']
    y0, y1 = year_selector.value
    upd_graph(p['st'], p['cty'], y0, y1)

def area_style(feature):
    style = dict(fillColor=feature['properties']['color'])
    return style
        
def refresh(_):
    s = state_selector.value
    y0, y1 = year_selector.value
    if len(map_.layers) == 2:
        map_.remove_layer(map_.layers[1])
    geo_json = json.loads(data_map(s, y0, y1, abs_rel_selector.value).to_json())
    l = leaflet.GeoJSON(data=geo_json,
                        style={'stroke': False, 'fillOpacity': 0.5},
                        hover_style={'stroke': True},
                        style_callback=area_style)
    l.on_click(click_geo)
    map_.add_layer(l)
    upd_graph(s, '000', y0, y1)

def upd_graph(st, cty, y0, y1):
    with graph:
        graph.clear_output(True)
        fig, ax = plt.subplots(3)
        plt.close()
        data_yr.query('st == @st and cty == @cty and year >= @y0 and year <= @y1').set_index('year')['pop'].plot(ax=ax[0])
        data_yr.query('st == @st and cty == @cty and year >= @y0 and year <= @y1').set_index('year')['emp'].plot(ax=ax[1])
        data_yr.query('st == @st and cty == @cty and year >= @y0 and year <= @y1').set_index('year')[['pop_gr', 'emp_gr']].plot(ax=ax[2])
        fig.suptitle(st)
        display(fig)

In [ ]:
year_selector = widgets.IntRangeSlider(value=(2007, 2012), min=1991, max=2019)
abs_rel_selector = widgets.RadioButtons(options=[('Absolute', 'abs'), ('Relative', 'rel')])
state_selector = widgets.Dropdown(options=data_yr['st'].unique())
refresh_button = widgets.Button(description='Refresh')
refresh_button.on_click(refresh)
controls = widgets.VBox([year_selector, abs_rel_selector, widgets.Label('Select US or state'), state_selector, refresh_button])

map_ = leaflet.Map(center=(40, -95), zoom=4)
refresh(None)

graph = widgets.Output()
upd_graph('00', '000', *year_selector.value)

widgets.AppLayout(left_sidebar=controls,
                center=map_, 
                right_sidebar=graph)